# Linear Algebra

After `using LinearAlgebra`, **Julia speaks linear algebra fluently**.

Performing linear algebra operations on a computer is, of course, an old problem. Lots of amazing libraries have been written - mostly in Fortran - which have been optimized over decades.

Basically all high-level programming languages use these libraries, including R, Python, and Julia.

Linear algebra in Julia is largely implemented by calling [BLAS](http://www.netlib.org/blas/)/[LAPACK](http://www.netlib.org/lapack/) functions. Sparse operations utilize functionality in [SuiteSparse](http://faculty.cse.tamu.edu/davis/suitesparse.html).

As per default, Julia uses the [OpenBLAS](https://github.com/xianyi/OpenBLAS) implementation (BLAS, LAPACK, LIBM), which can be replaced by [Intel's MKL](https://software.intel.com/en-us/mkl) (BLAS, LAPACK) and [Intel's Math Library](https://software.intel.com/en-us/node/522653) (LIBM).

**What is all this stuff?!?**

* **BLAS**: a collection of low-level matrix and vector arithmetic operations ("multiply two matrices", "multiply a matrix by vector").
* **LAPACK**:  a collection of higher-level linear algebra operations. Things like matrix factorizations (LU, LLt, QR, SVD, Schur, etc) that are used to do things like “find the eigenvalues of a matrix”, or “find the singular values of a matrix”, or “solve a linear system”.
* **LIBM**: basic math functions like `sin`, `cos`, `sinh`, etcetera

Sparse matrices are more difficult and there exist different collections of routines, one of which is **SuiteSparse**.

**Why do I have to care?**

* Switching from OpenBLAS to MKL can give you large speedups!
* Since you might be leaving the world of Julia code, you loose easy inspectability and type genericity. The latter can be an issue for machine learning, as we'll discuss later in more detail.

# Taking linear algebra seriously

Julia is [taking linear algebra seriously](https://www.youtube.com/watch?v=C2RO34b_oPM)! (see [here](https://github.com/JuliaLang/julia/issues/4774), and [here](https://github.com/JuliaLang/julia/issues/20978)).

In [ ]:
using LinearAlgebra

In [ ]:
A = rand(4,4)

In [ ]:
typeof(A)

In [ ]:
Array{Float64, 2} === Matrix{Float64} # equivalent not just equal

In [ ]:
det(A)

In [ ]:
inv(A)

In [ ]:
rank(A)

Let's get a vector as well

In [ ]:
v = rand(4)

In [ ]:
typeof(v)

In [ ]:
Array{Float64,1} === Vector{Float64}

In [ ]:
norm(v)

In [ ]:
v^2 # can't square a vector

In [ ]:
v.^2

Some things might be suprising

In [ ]:
1/v

But if it works, there is typically meaning to it. In this case it is calculating the [Moore-Penrose-Pseudoinverse](https://en.wikipedia.org/wiki/Moore%E2%80%93Penrose_inverse#Vectors) (`transpose(v)/sum(abs2,v)`).

### Identity matrix: `UniformScaling` operator

In [ ]:
A + 3

In [ ]:
A .+ 3

The `UniformScaling` operator **represents an identity matrix of any size** and is another great example of **duck typing**. It automatically gets loaded into scope when you do `using LinearAlgebra` and has the name `I`.

In [ ]:
I

Although it never actually materializes a full identity matrix it behaves like one.

In [ ]:
A + 3I

In [ ]:
I * A == A

Hence, we can calculate things like, say, `A-b*I` without ever allocating a dense identity matrix, which would take up $\mathcal{O}(n^2)$ memory.

Let's benchmark the performance difference!

In [ ]:
fullI = Matrix{Float64}(I, 4,4) # alternatively but slower, diagm(ones(4))

In [ ]:
fast(A) = A + 3*I

In [ ]:
slow(A, fullI) = A + 3*fullI

In [ ]:
function slower(A)
    fullI = Matrix(1.0I, size(A)...)
    A + 3*fullI
end

In [ ]:
using BenchmarkTools
@btime fast($A);
@btime slow($A, $fullI);
@btime slower($A);

# Fast linear algebra with multiple dispatch

Utilizing the structure of matrices by typing them appropriately!

In [ ]:
D = Diagonal(1:5)

In [ ]:
Ddense = Matrix(D) # same matrix but type doesn't indicate diagonal structure

In [ ]:
@btime $D*$b
@btime $Ddense*$b

What method does it dispatch to?

In [ ]:
@which D*b

In [ ]:
@which Ddense*b

**Dense Diagonal** (`Ddense*b`)

```julia
function (*)(A::AbstractMatrix{T}, x::AbstractVector{S}) where {T,S}
    TS = promote_op(matprod, T, S)
    mul!(similar(x,TS,axes(A,1)),A,x)
end
```

**Diagonal** (`D*b`)
```julia
(*)(D::Diagonal, V::AbstractVector) = D.diag .* V
```

There are a number of [special matrix](https://docs.julialang.org/en/v1/stdlib/LinearAlgebra/#Special-matrices) types are available out-of-the-box.

# Fermions hopping on a chain

$$\mathcal{H} = -t\sum_{\langle i,j \rangle} c_i^\dagger c_j + \mu \sum_i n_i$$

Here, $t$ is the hopping amplitude, $\mu$ is the chemical potential, and $c, c^\dagger$ are creation and annihilation operators.

For simplicity, we'll consider **open boundary conditions** (not periodic), in which case the Hamiltonian is tridiagonal.

Since the fermions are *not* interacting, we can work in the *single particle basis* and do not have to worry about how to construct a basis for the many-body Fock space.

We use the canonical cartesian basis in which one uses $0$s to indicate empty sites and a $1$ for the particle's site, i.e. $|00100\rangle$ represents the basis state which has the particle exclusively on the 3rd site.

If you aren't familiar with second quantization just think of $\mathcal{H}$ as any quantum mechanical operator that can be represented as a matrix.

In [ ]:
N = 100 # number of sites
t = 1
μ = -0.5

H = diagm(0 => fill(μ, N), 1 => fill(-t, N-1), -1 => fill(-t, N-1))

In [ ]:
ψ = normalize(rand(N)) # some state

Quantum mechanical expectation value

In [ ]:
ev(H, ψ) = ψ'*H*ψ # <φ|H|φ>

In [ ]:
ev(H, ψ)

In [ ]:
@btime ev($H, $ψ);

In [ ]:
typeof(H)

As long as the code is generic (respects the informal `AbstractArray` interface), we can use the same piece of code for completely different array types.

Let's utilize the sparsity of `H` by indicating it through a type.

In [ ]:
using SparseArrays
Hsparse = sparse(H)

In [ ]:
@btime ev($Hsparse, $ψ);

That's a solid **30x speedup**!

Our `H` isn't just sparse, but actually tridiagonal. Let's try to exploit that.

In [ ]:
Htri = Tridiagonal(H)

In [ ]:
@btime ev($Htri, $ψ);

Choosing the best type (and therewith an algorithm) can be tricky and one has to play around a bit. The good thing is that it's very easy to try out different types!

Note that there are also great matrix types available in the ecosystem, see [JuliaMatrices](https://github.com/JuliaMatrices), for example.

# Exact diagonalisation a.k.a Eigendecomposition

To diagonalize our dense "Hamiltonian", we simply call the built-in function `eigen`.

In [ ]:
vals, vecs = eigen(H)

Since Julia is using eigenproblem solvers from LAPACK (written in a low-level language) the code is, of course, **not generic**.

The best Julia can do, without implementing new functionality, is manually dispatch to the best LAPACK routine available.

Hence, it won't work with most of our special matrices.

In [ ]:
eigen(Htri);

If we're lucky, someone has implemented a generic solver in Julia that works for a wider range of types. Example:

In [ ]:
Hbig = big.(H)
eigen(Hermitian(Hbig));

In [ ]:
using GenericLinearAlgebra

In [ ]:
eigen(Hermitian(Hbig))

Arguably the most important matrix type in science applications is a sparse matrix, i.e. `SparseMatrixCSC`.

In [ ]:
eigen(Hsparse)

Let's follow Julia's advice and take a look at [ARPACK.jl](https://github.com/JuliaLinearAlgebra/Arpack.jl) and similar packages.

### Diagonalizing sparse matrices

[ARPACK.jl]() -  Wrapper to Fortran library [ARPACK](https://www.caam.rice.edu/software/ARPACK/) which implements **iterative** eigenvalue and singular value solvers. By far the most established sparse eigensolver.

Julia implementations:

* [ArnoldiMethod.jl](https://github.com/haampie/ArnoldiMethod.jl)
* [KrylovKit.jl](https://github.com/Jutho/KrylovKit.jl)
* [IterativeSolvers.jl](https://github.com/JuliaMath/IterativeSolvers.jl)
* and more


A key thing to remember is that while `eigen` is - up to numerical errors - exact, the methods in the packages above are iterative and approximative.

Arpack uses a different name for the eigenvalue decomposition. They called it `eigs`.

In [ ]:
using Arpack
λ, evs = eigs(Hsparse);
λ

In KrylovKit, they call the function `eigsolve`.

In [ ]:
using KrylovKit
λ, evs = eigsolve(Hsparse);
λ

# Core messages of this Notebook

* The standard libraries `LinearAlgebra` and `SparseArrays` make Julia speak linear algebra.
* **Indicate properties and structure of a matrix**, like hermiticity or sparsity, through types. Fallback to generic types only if you run into method errors.
* For **sparse matrix exact diagonalization**, ARPACK.jl is sort of a standard but there are great alternatives like KrylovKit.jl.